In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pysentiment2 as ps
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import yfinance as yf


def sentiment_vader(var):
    #pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
    sentiment = SentimentIntensityAnalyzer() 
    out_score = sentiment.polarity_scores(var)
    return out_score
def sentiment_ps(var):
    lm = ps.LM()
    tokens = lm.tokenize(var)
    score = lm.get_score(tokens)
    return score
def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
 
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(ldamodel[corpus])[0]
 
    return output
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
def normalize(arr):
     return (arr-min(arr))/(max(arr)-min(arr))
def normalize_2(arr):
     return np.interp(arr, (arr.min(), arr.max()), (-1, +1))

In [2]:
df = pd.read_pickle("sentiment.pkl")
df['Year']= pd.to_datetime(df['Year'])
df = df.set_index('Year')
df['Year'] = df.index

In [6]:
# Get Stock Market Data: S&P Mid-Cap 400 Index
import yfinance as yf
stock_info = yf.download("^MID", start="1993-02-01", end="2021-02-20")
stock_info.index = pd.to_datetime(stock_info.index)
stock_info

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-02-01,162.759995,163.520004,162.350006,163.500000,163.500000,0
1993-02-02,163.550003,164.309998,163.419998,164.309998,164.309998,0
1993-02-03,164.839996,165.580002,164.309998,165.449997,165.449997,0
1993-02-04,165.449997,166.779999,165.449997,166.759995,166.759995,0
1993-02-05,166.490005,166.759995,165.089996,165.320007,165.320007,0
1993-02-08,165.479996,165.600006,164.940002,165.020004,165.020004,0
1993-02-09,164.619995,165.110001,163.470001,163.729996,163.729996,0
1993-02-10,163.899994,164.149994,163.419998,164.149994,164.149994,0
1993-02-11,164.259995,164.449997,163.929993,163.990005,163.990005,0


In [33]:
# stock market volatibility within 4 days after the Fed meetings, window = 4 performs the best
import numpy as np
price_changes = []
price_change_percentages = []
labels = []
window = 4 
# workaround for rolling widow
for index, row in df.iterrows():
    minutes_date = row['Year']
    try:
        start = stock_info.index.get_loc(minutes_date)
    except:
        start = stock_info.index.get_loc(minutes_date + pd.DateOffset(days=1))
    stock_closing_sum = stock_info.iloc[start+window]['Close']
    start_price = stock_info['Close'].iloc[start]
    price_change = stock_closing_sum - start_price
    price_change_percent = (stock_closing_sum - start_price) / start_price
    price_changes.append(price_change)
    price_change_percentages.append(price_change_percent)
    

df['price_change'] = price_changes
df['price_change_percent'] = price_change_percentages

#label =1 means high volatility, label = 1 means low volatility. High volatility is identified if VIX price change percent is far from its median (greater than 75% or smaller than 25%)
df['label'] = df['price_change_percent'].apply(lambda x: 1 if x> df["price_change_percent"].quantile(0.75) or x < df["price_change_percent"].quantile(0.25) else 0)

In [34]:
X = df
X = np.array(X.drop(['label','Statement','Year','positive','negative','price_change','price_change_percent'],1))
y = np.array(df['label'])

In [35]:
#Split(80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.4, random_state = 0)

In [36]:
# Create and train model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [37]:
predictions = model.predict(x_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.54      0.83      0.65        42
           1       0.68      0.33      0.45        45

    accuracy                           0.57        87
   macro avg       0.61      0.58      0.55        87
weighted avg       0.61      0.57      0.55        87



In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [39]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
m = randomclassifier.fit(x_train,y_train)
predictions_2 = m.predict(x_test)
score = accuracy_score(y_test,predictions)
report = classification_report(y_test,predictions)
print(score)
print(report)

0.5747126436781609
              precision    recall  f1-score   support

           0       0.54      0.83      0.65        42
           1       0.68      0.33      0.45        45

    accuracy                           0.57        87
   macro avg       0.61      0.58      0.55        87
weighted avg       0.61      0.57      0.55        87

